In [ ]:
import cv2  # Import open cv2 library
import copy  # Import copy library
import numpy as np  # Import numpy library
import matplotlib.pyplot as plt  # Import matplotlib library
from imutils import paths # Import imutils library
from numpy.fft import fft2, ifft2 # Import fft and ifft modules
from scipy.signal.windows import gaussian # import gaussian module
from google.colab.patches import cv2_imshow #import cv2.im_show module

from google.colab import drive
drive.mount('/content/drive')



def scale(img,per):
    scale_percent = per  # percent of original size
    width = int(img.shape[1] * scale_percent / 100)  # scaling width of main image
    height = int(img.shape[0] * scale_percent / 100)  # scaling height of main image
    dim = (width, height) # dimensions of scaled image
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)  # resizing night image
    return img


def colored_equ(img):
    b_n, g_n, r_n = cv2.split(img) # splitting the image in 3 (blue,green,red) channels
    equ_b_nit = cv2.equalizeHist(b_n) # equalizing blue channel histogram
    equ_g_nit = cv2.equalizeHist(g_n) # equalizing green channel histogram
    equ_r_nit = cv2.equalizeHist(r_n) # equalizing red channel histogram
    equalised = cv2.merge((equ_b_nit, equ_g_nit, equ_r_nit)) # merging three equalised histogram channels into one image
    return equalised # returning of equalised colour image


def gaussian_kernel(kernel_size): # gaussian function implementation for weiner filtering 
    h = gaussian(kernel_size, kernel_size / 3) #  giving arguments to gaussian funtion where kernel_size=n, kernel_size/3=sigma.
    h = h.reshape(kernel_size, 1) # reshaping h matrix to (kernel_size,1) matrix
    h = np.dot(h, h.transpose()) # matrix multiplication of h with h traanspose
    h /= np.sum(h) # h=h/(sum of all elements of h matrix)
    return h # returning h matrix


def wiener_filter(img, kernel1, k):
    kernel1 /= np.sum(kernel1) # kernel1=kernel1/(sum of all elements of kernel1 matrix)
    dummy = np.copy(img) # making a copy of original image
    dummy = np.fft.fft2(dummy) # computing fast fourier transform of given image
    kernel1 = np.fft.fft2(kernel1,s=img.shape) # computing fast fourier transform of kernel1 
    kernel1 = np.conj(kernel1) / (np.abs(kernel1) ** 2 + k) # according to formula given in slides for weiner filering
    dummy = dummy * kernel1
    dummy = np.abs(ifft2(dummy)) # inverse fourier transform of modified  image
    return dummy # returning final output


def contrast(grey):
    mini = np.min(grey)  # computing minimum intesity  level of grey scale image
    maxi = np.max(grey)  # computing maximum intesity  level of grey scale image
    contrast_img = (int(maxi)-int(mini))/(int(maxi)+int(mini)) # computing contrast
    return mini, maxi, float(contrast_img) # returning required values


def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var() # calculating laplacian of variance for quantification of bluriness


def blur(gray):
    fm = variance_of_laplacian(gray)  # calculates threshold value
    text = "Not Blurry"
    # if the focus measure is less than the supplied threshold,
    # then the image should be considered "blurry"

    if fm < 150:
        text = "Blurry"
    return fm,text


def horizontal_notch_reject(shape): 
    p,q=shape # dimensions of image
    H = np.zeros((p,q)) # zeros matrix
    for u in range(0, p):
        for v in range(0, q):
            if p/2-p/100<=u<=p/2+p/100  and v<=q and np.sqrt((u - p/2) ** 2 + (v - q/2) ** 2) > 5: # drawing rectangular band reject filter  with circular gap at its center
                H[u,v]=0
            else:
                H[u,v]=1
    return H # returning band rejected matrix


def notch_reject_filter(shape, d0=9, u_k=0, v_k=0): # drawing circular band reject around required point gives output in frquency domain as 0
    P, Q = shape # dimensions of image
    H = np.zeros((P, Q)) # zeros matrix
    for u in range(0, P):
        for v in range(0, Q):
            D_uv = np.sqrt((u - P / 2 + u_k) ** 2 + (v - Q / 2 + v_k) ** 2) # condition for circle around a required point
            D_muv = np.sqrt((u - P / 2 - u_k) ** 2 + (v - Q / 2 - v_k) ** 2)
            if D_uv <= d0 or D_muv <= d0:
                H[u, v] = 0.0 # output 0 for points lying inside the circle 
            else:
                H[u, v] =1.0 # output 1 for points lying outside the circle
    return H # returning band reject matrix



def filtering(voyager): 
    f = np.fft.fft2(voyager) # calculating fourier transform of given image
    fshift = np.fft.fftshift(f) # shifting frequency domain to center of plot 
    magnitude_spectrum = 20 * np.log(np.abs(fshift)) # magnitude spectrum of fft of image
    notch_rej = fshift * horizontal_notch_reject(voyager.shape) # multiplying centered fft of image with notch reject filter in frequency domain
    final = np.fft.ifftshift(notch_rej) # centralising inverse fourier transform of output image
    final = np.fft.ifft2(final) # inverse fourier transform of output image
    final = np.abs(final) # taking absolute value of final output
    plt.imshow(magnitude_spectrum, cmap='gray') # magnitude spectrum of fft as output
    plt.show()
    plt.imshow(magnitude_spectrum * horizontal_notch_reject(voyager.shape), cmap='gray')  # modified  frequency domain plot after multiplicaion of two components
    plt.show()
    plt.imshow(final) # final image
    plt.show()
    return final


def filtering1(dot_img):
  f1 = np.fft.fft2(dot_img) # calculating fourier transform of given image
  fshift1= np.fft.fftshift(f1) # shifting frequency domain to center of plot 
  magnitude_spectrum1 = 20*np.log(np.abs(fshift1)) # magnitude spectrum of fft of image
  img_shape = dot_img.shape
  H1 = notch_reject_filter(img_shape, 4, 58, 37) # circular dot centred at 58,37 points inside this circle goes to zero
  H2 = notch_reject_filter(img_shape, 4, -52, 35) # circular dot centred at -52,35 points inside this circle goes to zero
  H3 = notch_reject_filter(img_shape, 2, 80, 30) # circular dot centred at 80,32 points inside this circle goes to zero
  H4 = notch_reject_filter(img_shape, 2, -82, 28) # circular dot centred at -82,28 points inside this circle goes to zero
  NotchFilter = H1*H2*H3*H4 # multiplying all the notch filters in frequency domain
  NotchRejectCenter = fshift1 * NotchFilter  # multiplying centered fft of image with notch reject filter in frequency domain
  NotchReject = np.fft.ifftshift(NotchRejectCenter) # centralising inverse fourier transform of output image
  inverse_NotchReject = np.fft.ifft2(NotchReject) # inverse fourier transform of output image
  Result = np.abs(inverse_NotchReject) # taking absolute value of final output
  plt.imshow(magnitude_spectrum1, cmap='gray') # magnitude spectrum of fft as output
  plt.show()
  plt.imshow(magnitude_spectrum1*NotchFilter, "gray")  # modified  frequency domain plot after multiplicaion of two components
  plt.show()

  return Result


#  reading images
nit_img = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/IMG_20220901_201400.jpg')
dot_img = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/newspaper-dots.jpg')
voyager = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/triton_voyager2.jpg')
car = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/MakeNumberPlateReadable.jpg')
dot_img = cv2.cvtColor(dot_img, cv2.COLOR_BGR2GRAY)


# scaling images
nit_img = scale(nit_img, 15)
dot_img = scale(dot_img, 100)
voyager = scale(voyager, 100)

#  equalising images histogram
equ_nit = colored_equ(nit_img)
dot_img = cv2.equalizeHist(dot_img)


#  de-noising images
nit_noiseless = cv2.fastNlMeansDenoisingColored(equ_nit, None, 5, 5, 7, 21) # denoising using fastnl means denoising algorithm
dot_noiseless = cv2.medianBlur(dot_img, 7) # median blurring of given image
dot_noiseless = cv2.fastNlMeansDenoising(dot_noiseless, None, 15, 15, 21) # denoising using fastnl means denoising algorithm

# removing Dots from newspaper 2nd method
dot_dotless=filtering1(dot_img) # notch filtering of given image

# voyager removing scan lines
b_v, g_v, r_v = cv2.split(voyager) #spliting of image in three  channels
b_v = filtering(b_v) # applying horizontal notch filter to blue channel seperately
g_v = filtering(g_v) # applying horizontal notch filter to green channel seperately
r_v = filtering(r_v) # applying horizontal notch filter to red channel seperately
merged_voyager=cv2.merge((b_v,g_v,r_v)) # merging all channels into one

#sharpening of image
kernel1 = np.array([[0,-1,0], # sharpening matrix using laplacian filter
                   [-1,5,-1],
                   [0,-1,0]]) 
img_sharp = cv2.filter2D(car,ddepth=-1,kernel=kernel1)

#  weiner filtering
kernel = gaussian_kernel(3) # calculting gaussian values at each position
b_c,g_c,r_c=cv2.split(car) #splitting images into three channels
b_c = wiener_filter(b_c, kernel,0.5) # applying weiner filtering to blue channel
g_c = wiener_filter(g_c, kernel,0.5) # applying weiner filtering to green channel
r_c = wiener_filter(r_c, kernel,0.5) # applying weiner filtering to red channel
merged_car=cv2.merge((b_c,g_c,r_c))  # merging all three channels into one

'''plt.imshow(magnitude_spectrum, cmap='gray')
plt.show()'''

#   ML based restoring:


#   Ml images reading
bi1 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/Screenshot_2022_0911_104252.png')
bi2 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/Screenshot_2022_0911_104215.png')
bi3 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/Screenshot_2022_0911_104142.png')
bi4 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/IMG_20220826_215838.jpg')
bi5 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/IMG_20220826_215842.png')
bi6 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/IMG_20220809_193235.jpg')
bi7 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/IMG_20220815_005116.jpg')
bi8 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/bad_pics/IMG_20220809_193235.jpg')
gi1 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/good_pics/training_set/IMG_20220831_214444.jpg')
gi2 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/good_pics/training_set/gandhi.jpg')
gi3 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/good_pics/training_set/IMG_20220815_022429.jpg')
gi4 = cv2.imread('/content/drive/MyDrive/ee610_ass2_pics/good_pics/training_set/IMG_20220904_011934.jpg')
#converting into gray scale images
bi1 = cv2.cvtColor(bi1, cv2.COLOR_BGR2GRAY)
bi2 = cv2.cvtColor(bi2, cv2.COLOR_BGR2GRAY)
bi3 = cv2.cvtColor(bi3, cv2.COLOR_BGR2GRAY)
bi4 = cv2.cvtColor(bi4, cv2.COLOR_BGR2GRAY)
bi5 = cv2.cvtColor(bi5, cv2.COLOR_BGR2GRAY)
bi6 = cv2.cvtColor(bi6, cv2.COLOR_BGR2GRAY)
bi7 = cv2.cvtColor(bi7, cv2.COLOR_BGR2GRAY)
bi8 = cv2.cvtColor(bi8, cv2.COLOR_BGR2GRAY)
gi1 = cv2.cvtColor(gi1, cv2.COLOR_BGR2GRAY)
gi2 = cv2.cvtColor(gi2, cv2.COLOR_BGR2GRAY)
gi3 = cv2.cvtColor(gi3, cv2.COLOR_BGR2GRAY)
gi4 = cv2.cvtColor(gi4, cv2.COLOR_BGR2GRAY)

#   scaling images
bi1=scale(bi1,30)
bi2=scale(bi2,30)
bi3=scale(bi3,30)
bi4=scale(bi4,30)
bi5=scale(bi5,30)
bi6=scale(bi6,30)
bi7=scale(bi7,30)
bi8=scale(bi8,30)

gi1=scale(gi1,30)
gi2=scale(gi2,30)
gi3=scale(gi3,30)
gi4=scale(gi4,30)
#   Blurriness images checking:
print('Variance value and blurriness of bad image 1 are : ',blur(bi1))
print('Variance value and blurriness of bad image 2 are : ',blur(bi2))
print('Variance value and blurriness of bad image 3 are : ',blur(bi3))
print('Variance value and blurriness of bad image 4 are : ',blur(bi4))
print('Variance value and blurriness of bad image 5 are : ',blur(bi5))
print('Variance value and blurriness of bad image 6 are : ',blur(bi6))
print('Variance value and blurriness of bad image 7 are : ',blur(bi7))
print('Variance value and blurriness of bad image 8 are : ',blur(bi8))
print('Variance value and blurriness of good image 1 are : ',blur(gi1))
print('Variance value and blurriness of good image 2 are : ',blur(gi2))
print('Variance value and blurriness of good image 3 are : ',blur(gi3))
print('Variance value and blurriness of good image 4 are : ',blur(gi4))
# average brightness
print('average brightness of bad image 1 is : ', np.average(bi1))
print('average brightness of bad image 2 is : ', np.average(bi2))
print('average brightness of bad image 3 is : ', np.average(bi3))
print('average brightness of bad image 4 is : ', np.average(bi4))
print('average brightness of bad image 5 is : ', np.average(bi5))
print('average brightness of bad image 6 is : ', np.average(bi6))
print('average brightness of bad image 7 is : ', np.average(bi7))
print('average brightness of bad image 8 is : ', np.average(bi8))
print('average brightness of good image 1 is : ', np.average(gi1))
print('average brightness of good image 2 is : ', np.average(gi2))
print('average brightness of good image 3 is : ', np.average(gi3))
print('average brightness of good image 4 is : ', np.average(gi4))
#  contrast values checking of  bad and good images:
print('Minimum , Maximum intensity level and contrast of 1st bad image:',contrast(bi1))
print('Minimum , Maximum intensity level and contrast of 2nd bad image:',contrast(bi2))
print('Minimum , Maximum intensity level and contrast of 3rd bad image:',contrast(bi3))
print('Minimum , Maximum intensity level and contrast of 4th bad image:',contrast(bi4))
print('Minimum , Maximum intensity level and contrast of 5th bad image:',contrast(bi5))
print('Minimum , Maximum intensity level and contrast of 6th bad image:',contrast(bi6))
print('Minimum , Maximum intensity level and contrast of 7th bad image:',contrast(bi7))
print('Minimum , Maximum intensity level and contrast of 8th bad image:',contrast(bi8))
print('Minimum , Maximum intensity level and contrast of 1st good image:',contrast(gi1))
print('Minimum , Maximum intensity level and contrast of 2nd good image:',contrast(gi2))
print('Minimum , Maximum intensity level and contrast of 3rd good image:',contrast(gi3))
print('Minimum , Maximum intensity level and contrast of 4th good image:',contrast(gi4))
#  tweaking good images to distort them by applying gaussian blur to all and then power transform for contrast changing :

gci1 = copy.copy(gi1)
gci2 = copy.copy(gi2)
gci3 = copy.copy(gi3)
gci4 = copy.copy(gi4)
gci1 = cv2.GaussianBlur(gci1,(5,5),cv2.BORDER_DEFAULT)
gci2 = cv2.GaussianBlur(gci2,(5,5),cv2.BORDER_DEFAULT)
gci3 = cv2.GaussianBlur(gci3,(5,5),cv2.BORDER_DEFAULT)
gci4 = cv2.GaussianBlur(gci4,(5,5),cv2.BORDER_DEFAULT)
gci1 = np.array(255*((gci1/255)**0.4), dtype=np.uint8)
gci2 = np.array(255*((gci2/255)**0.4), dtype=np.uint8)
gci3 = np.array(255*((gci3/255)**0.4), dtype=np.uint8)
gci4 = np.array(255*((gci4/255)**0.4), dtype=np.uint8)

print('Variance value and blurriness of good distorted image 1 are : ',blur(gci1))
print('Variance value and blurriness of good distorted image 2 are : ',blur(gci2))
print('Variance value and blurriness of good distorted image 3 are : ',blur(gci3))
print('Variance value and blurriness of good distorted image 4 are : ',blur(gci4))

print('average brightness of good distorted image 1 is : ', np.average(gci1))
print('average brightness of good distorted image 2 is : ', np.average(gci2))
print('average brightness of good distorted image 3 is : ', np.average(gci3))
print('average brightness of good distorted image 4 is : ', np.average(gci4))

print('Minimum , Maximum intensity level and contrast of 1st good distorted image:',contrast(gci1))
print('Minimum , Maximum intensity level and contrast of 2nd good distorted image:',contrast(gci2))
print('Minimum , Maximum intensity level and contrast of 3rd good distorted image:',contrast(gci3))
print('Minimum , Maximum intensity level and contrast of 4th good distorted image:',contrast(gci4))
#  showing output
cv2_imshow( nit_noiseless) # night noiseless image with increased brightness and  clarity
cv2_imshow(dot_noiseless) # dot less image of newspaper man using median blurring
cv2_imshow(img_sharp) # sharpened image of car 
cv2_imshow(merged_voyager) # horizontal notch filtered image of voyager
cv2_imshow(merged_car) # weiner filtered image 
cv2_imshow(dot_dotless) # notch filtered dotless image
#  Saving pic locally
cv2.imwrite('nit noiseless.png', nit_noiseless, [cv2.IMWRITE_PNG_COMPRESSION])
cv2.imwrite('dot noiseless.png', dot_noiseless, [cv2.IMWRITE_PNG_COMPRESSION])

#  closing arguments
cv2.waitKey(0)  # waiting till i close it
cv2.destroyAllWindows()  # destroying all windows after stopping run

''' sources:
https://mathworld.wolfram.com/GaussianFunction.html
https://docs.opencv.org/3.4/de/dbc/tutorial_py_fourier_transform.html
https://stackoverflow.com/questions/65483030/notch-reject-filtering-in-python
https://stackoverflow.com/questions/55288657/image-is-not-displaying-in-google-colab-while-using-imshow
https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/#:~:text=Syntax%20%E2%80%93%20cv2%20GaussianBlur()%20function&text=%5Bheight%20width%5D.,is%20computed%20from%20sigma%20values.&text=Kernel%20standard%20deviation%20along%20X%2Daxis%20(horizontal%20direction).
https://pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
https://www.analyticsvidhya.com/blog/2021/08/sharpening-an-image-using-opencv-library-in-python/
https://www.bogotobogo.com/python/OpenCV_Python/python_opencv3_Image_Non-local_Means_Denoising_Algorithm_Noise_Reduction.php
https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html

'''
